In [107]:
# pip install customtkinter

In [108]:
from tkinter import *
import tkinter as tk
from tkinter.filedialog import askopenfilename
#NOVO
import customtkinter as s
import os
from PIL import Image, ImageTk
import pm4py
from pm4py.objects.petri_net.importer import importer as pmnl_importer
import pandas as pd


## Funções

### função que escolhe o caminho 

In [109]:

def escolhe_arquivo():
    global log
    caminho_do_arquivo = askopenfilename(filetypes=[ ("Arquivo Excel", "*.xlsx"),("Arquivo CSV", "*.csv"), ("Arquivo XES", '*.xes')])
    if caminho_do_arquivo:
        label_arquivo.configure(text=f"Arquivo selecionado: {caminho_do_arquivo}") # Calma é pra não estar definido msm
        
####################################### ANALISA O TIPO DO ARQUIVO #######################################
    tipoArq = caminho_do_arquivo.split('.')
    try:
        if tipoArq[-1] in 'csv':
            log = pm4py.format_dataframe(pd.read_csv(caminho_do_arquivo, sep= ','), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xlsx':
            log = pm4py.format_dataframe(pd.read_excel(caminho_do_arquivo), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

        elif tipoArq[-1] in 'xes':
            log = pm4py.format_dataframe(pm4py.read_xes(caminho_do_arquivo), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 
    except Exception as error:
        lable_auxImagem.configure(text=f"NÃO CONSEGUI TRANSFORMAR EM GRÁFICO. UMA COLUNA NÃO FOI ENCONTRADA NESSE ARQUIVO.\n erro: {error}")

    
    return log

### FILTRO

In [110]:
def filtra_grafo_por_variantes():

    log.fillna({'Início': 0},inplace=True)
    log['Início'] = pd.to_datetime(log['Início'])
    log['ID'] = log['ID'].astype(str)

    filtrado = pm4py.filtering.filter_variants_by_coverage_percentage(log, paths.get(), case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
    filtrado = filtrado.drop('Unnamed: 0', axis=1)
    return filtrado

### função cria gráfico frequência (EM TESTE)

In [111]:
def cria_grafo_dfg(caminho):
############################################################################## FILTRA OS PATHS ##############################################################################
    filtrado = filtra_grafo_por_variantes()
#############################################################################################################################################################################
    dfg, start_activities, end_activities = pm4py.discover_dfg(filtrado)
    pm4py.vis.save_vis_dfg(dfg, start_activities, end_activities, "grafico.png")#, rankdir='TB'
    
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do DFG salva como 'grafico.png'")

    ##---------------------------------------------------------------DEIXEI PRA DEPOIS-------------------------------------------------------------------------

    exibe_grafico("grafico.png")
    return

#### função cria grafico de performance

In [112]:
def cria_grafo_duracao(caminho):
############################################################################## FILTRA OS PATHS ##############################################################################
    filtrado = filtra_grafo_por_variantes()
#############################################################################################################################################################################

    performance_dfg, start_activities, end_activities = pm4py.discover_performance_dfg(filtrado, case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
######################################################## ESSE PARAMETRO COLOCA NO GRÁFICO O TEMPO QUE CADA ATIVIDADE DUROU ##################################################
    
    #########serv_time – (optional) provides the activities’ service times, used to decorate the graph
    
    pm4py.vis.save_vis_performance_dfg(performance_dfg, start_activities, end_activities, "grafico_Duracao.png") #, rankdir='TB' #bgcolor='str' tem como escolher a cor de fundo
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do DFG salva como 'grafico_Duracao.png'")

    exibe_grafico("grafico_Duracao.png")
    return

#### função cria petri net

In [113]:
def cria_grafo_petri_net(caminho):
    ##----------------------------------------------------------------------------------------------------------------------------------------
 
    pn, ini, fim = pm4py.discover_petri_net_inductive(log)
    pm4py.vis.save_vis_petri_net(pn, ini, fim, "grafico_PetriNet.png")#, rankdir='TB'
    ##----------------------------------------------------------------------------------------------------------------------------------------
    lable_auxImagem.configure(text="Imagem do rede de Petri salva como 'grafico_PetriNet.png'")




    exibe_grafico("grafico_PetriNet.png")
    return

### Função que vai plotar os gráficos

In [114]:
lableimg_atual = None

def exibe_grafico(caminho):
    global lableimg_atual
    global imagem
    caminho = './'+caminho
    imagem = Image.open(caminho)

    # Define o tamanho fixo para todos os gráficos
    tamanho_grafico_largura = 1340 
    tamanho_grafico_altura = 110

    # Redimensione a imagem do gráfico para o tamanho fixo
    imagem_redimensionada = imagem.resize((tamanho_grafico_largura, tamanho_grafico_altura))
    # Remova o label antigo, se existir
    if lableimg_atual:
        lableimg_atual.destroy()

    # Crie um novo label para o gráfico
    fig_grafico = s.CTkImage(light_image=imagem_redimensionada, dark_image=imagem_redimensionada, size= (tamanho_grafico_largura, tamanho_grafico_altura))
    lableimg_atual = s.CTkLabel(frame_grafico, text=None, image=fig_grafico)
    lableimg_atual.pack(side='top')
    
    return get_variantes()


In [115]:
def analize_conformidade():
    log['ID'] = log['ID'].astype(str)
    caminho_do_arquivo = askopenfilename(filetypes=[("Arquivo Pnml", "*.pnml")])
    rede, inicial, final = pmnl_importer.apply(caminho_do_arquivo)
    # declare_model = pm4py.discover_declare(declare_model22)
    conf_result = pm4py.fitness_token_based_replay(log, rede, inicial, final, case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
    lableConform.configure(text=f'mean fittness: {conf_result["average_trace_fitness"]} \n percentage fittness {conf_result["percentage_of_fitting_traces"]}')
    return

In [116]:
# Função para alternar o modo de aparência
def toggle_mode():
    state = switch.get()
    if state == "on":
        s.set_appearance_mode("Light")
    else:
        s.set_appearance_mode("Dark")


# _________________________________________________________________________________________________

# TESTES

### adicionando variantes

In [117]:
def get_variantes():
    variants = pm4py.get_variants_as_tuples(log)
    # labelVariantes.configure(text= variants)
    return

### Analize de conformidade

In [118]:
# ANÁLISE DE CONFORMIDADE --- DUDA #

def analiseConformidadeFALSA(log):
    # Para o modelo não ser igual ao normal (TESTE)
    declare_model22= pm4py.filtering.filter_variants_by_coverage_percentage(log, 0.5)

    declare_model = pm4py.discover_declare(declare_model22)
    conf_result = pm4py.conformance_declare(log, declare_model)

    for cada in conf_result:
        if cada['deviations'] != [] and cada['dev_fitness'] != 0.9135802469135803:
            print(cada['deviations'], '\n')
            print(cada['dev_fitness'])
            print('\n')


### Filtrando os gráficos (Paths) (CONCLUÍDO)

In [119]:
# def filtra_grafo_por_variantes(caminho):
# ###################################################################################################################################################################################################
#     tipoArq = caminho.split('.')
#     try:
#         if tipoArq[-1] in 'csv':
#             log = pm4py.format_dataframe(pd.read_csv(caminho, sep=','), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

#         elif tipoArq[-1] in 'xlsx':
#             log = pm4py.format_dataframe(pd.read_excel(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id activity_key e timestamp_key

#         elif tipoArq[-1] in 'xes':
#             log = pm4py.format_dataframe(pm4py.read_xes(caminho), case_id='ID', activity_key='Ocorrência', timestamp_key='Início') # depois arrumar um jeito de o usuario selecionar as colunas case_id 
#     except Exception as error:
#         print("NÃO CONSEGUI TRANSFORMAR EM GRÁFICO. UMA COLUNA NÃO FOI ENCONTRADA NESSE ARQUIVO.\n erro: {error}")

# ###################################################################################################################################################################################################

#     log.fillna({'Início': 0},inplace=True)
#     log['Início'] = pd.to_datetime(log['Início'])
#     log['ID'] = log['ID'].astype(str)

#     filtrado = pm4py.filtering.filter_variants_by_coverage_percentage(log, paths.get(), case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
#     filtrado = filtrado.drop('Unnamed: 0', axis=1)

# ###################################################################################################################################################################################################

#     dfg, start_activities, end_activities = pm4py.discover_performance_dfg(filtrado, case_id_key='ID', activity_key='Ocorrência', timestamp_key='Início')
#     pm4py.vis.save_vis_performance_dfg(dfg, start_activities, end_activities, "grafico_Duracao_filtrado.png")
    
#     ##----------------------------------------------------------------------------------------------------------------------------------------
#     lable_auxImagem.configure(text="Imagem do DFG salva como 'grafico_Duracao_filtrado.png'")

#     exibe_grafico("grafico_Duracao_filtrado.png")

### Função que pega valor do widget Scale (função TESTE)

In [120]:
# def pega_valor_scale():
#     print('atividades:')
#     print(escala1.get())
#     print('Paths')
#     print(paths.get())
#     print(log)

In [121]:
# def cria_grafo_analise(caminho):
    
# def filter_dfg_on_activities_percentage(dfg0, start_activities0, end_activities0, activities_count0, percentage):
# def filter_dfg_on_paths_percentage(dfg0, start_activities0, end_activities0, activities_count0, percentage,
#                                    keep_all_activities=False):

### (TESTE) redefinindo o tamanho da imagme com o tamanho da janela

In [122]:
################## AINDA NÃO DEU CERTO ##################
# def resize_image(event):
#     new_width = event.width
#     new_height = event.height
#     image = imagem.resize((new_width, new_height))
#     photo = Image.open(image)
#     lableimg_atual.config(image=photo)
#     lableimg_atual.image = photo

# _________________________________________________________________________________________________

## Instanciando a janela

In [123]:
raiz = s.CTk(className='Proces Mining GUI')
s.set_appearance_mode("system")  # "dark", "light", "system"
s.set_default_color_theme("dark-blue")  # "blue", "dark-blue", "green"
raiz.geometry('960x540')

#---------------------------------------------TENTATIVA PNG---------------------------------------------------------
# raiz.title("Janela com Ícone Personalizado")

# # Carregar a imagem do ícone usando PIL
# image = Image.open('C:\\Users\\meduarda\\grupo-2-7\\Notebooks\\Tkinter_resumo\\iconProcessGUI.png')
# icon_image = ImageTk.PhotoImage(image)

# # Definir a imagem como ícone da janela
# raiz.iconphoto(False, icon_image)
#---------------------------------------------TENTATIVA .ICO---------------------------------------------------------

# image = Image.open("iconProcessGUI.png")
# icon_image = ImageTk.PhotoImage(image)

# # Definir a imagem como ícone da janela
# raiz.iconphoto(False, icon_image)

## Frames

### Criando o frame de seleção de arquivos

In [124]:
frame1 = s.CTkFrame(raiz)
frame1.pack(side='top',fill=tk.BOTH)

In [125]:
switch_var = s.StringVar(value="off")
switch = s.CTkSwitch(frame1, text=None, variable=switch_var, onvalue="on", offvalue="off", command=lambda: toggle_mode())
switch.pack(side='right')

### Criando o frame Lateral

In [126]:
frame_lateral = s.CTkFrame(raiz)
frame_lateral.pack(side = 'right',pady=0,  fill='y')

### Frame do grafico

In [127]:
frame_grafico = s.CTkFrame(raiz)
frame_grafico.pack(expand=True, fill='both')
# frame_grafico.bind("<Configure>", resize_image)

In [128]:
frameConform = s.CTkFrame(raiz)
frameConform.pack(side='right', fill='both')

## Imagens

In [129]:
arq = s.CTkImage(light_image=Image.open('iconArq.png'), dark_image=Image.open('iconArq.png'), size=(80,80))
# arq = arq.subsample(2,2)

arqPerform = s.CTkImage(light_image=Image.open('iconPerformance.png'), dark_image=Image.open('iconPerformanceDark.png'), size=(80,80))
# arqPerform = arqPerform.subsample(5,5)

arqFreq = s.CTkImage(light_image=Image.open('iconFreq.png'), dark_image=Image.open('iconFreqDark.png'), size=(80,80))
# arqFreq = arqFreq.subsample(5,5)

arqPetri = s.CTkImage(light_image=Image.open('iconPetri.png'), dark_image=Image.open('iconPetriDark.png'), size=(80,80))
# arqPetri = arqPetri.subsample(5,5)

arqNot = s.CTkImage(light_image=Image.open('mode.png'), dark_image=Image.open('modeDark.png'), size=(20,20))

## Widgets

### Botão de seleção de arquivos ###

In [130]:
label_arquivo = s.CTkLabel(frame1, text="Nenhum arquivo selecionado")
label_arquivo.pack(side='bottom')

lable_auxImagem= s.CTkLabel(frame1, text="Nenhma Imagem Gerada")
lable_auxImagem.pack(side='bottom')

### Botões de seleção de Gráfico (TESTE) ###

In [131]:

botaoArquivo = s.CTkButton(frame1, text= "Seleção de arquivos" , image=arq, command=lambda: escolhe_arquivo(), hover_color=None, fg_color="black")
botaoArquivo.pack(side='left')

botaoFrequencia = s.CTkButton(frame1, text="Gráfico de Frequência", image=arqFreq, command=lambda: cria_grafo_dfg(log), hover_color=None, fg_color="transparent")
botaoFrequencia.pack(side='left', pady=10)

botaoPerformance = s.CTkButton(frame1, text="Gráfico de Performace", image=arqPerform,  command=lambda: cria_grafo_duracao(log), hover_color=None, fg_color="transparent")
botaoPerformance.pack(side='left', pady=10)

botaoPetriNet = s.CTkButton(frame1, text="Gráfico PetriNet", image=arqPetri,  command=lambda: cria_grafo_petri_net(log), hover_color=None, fg_color="transparent")
botaoPetriNet.pack(side='left', pady=10)

# toggle_button = s.CTkButton(frame1, text= None, image=arqNot, command=toggle_mode, hover_color=None, fg_color="transparent")
# toggle_button.pack(side='right')

In [132]:

# Criando A lable na lateral direita
# label_direita_sup = s.CTkLabel(frame_lateral)#, padx=40, pady=10, borderwidth=2, relief="solid"
# label_direita_sup.pack(fill=tk.Y, expand=True)

################################################## DUDA
# Criando um frame para cada conjunto de label e escala
frame_escala1 = s.CTkFrame(frame_lateral)
frame_escala1.pack(fill="both", expand=True)

# frame_escala2 = s.CTkFrame(frame_lateral, bg_color='green')
# frame_escala2.pack(side = 'top')

# Adicionando rótulo e escala no primeiro frame
label_escala1 = s.CTkLabel(frame_escala1, text="Activities")
label_escala1.place(relx=0.3, rely=0.1, anchor=tk.CENTER)

escala1 = s.CTkSlider(frame_escala1,from_=0, to=1, orientation='vertical')

escala1.set(100)
escala1.place(relx=0.3, rely=0.5, anchor=tk.CENTER)

# Adicionando rótulo e escala no segundo frame
label_escala2 = s.CTkLabel(frame_escala1, text="Paths")
label_escala2.place(relx=0.7, rely=0.1, anchor=tk.CENTER)

paths = s.CTkSlider(frame_escala1, from_=0.95, to=0, orientation='vertical')
paths.set(0)
paths.place(relx=0.7, rely=0.5, anchor=tk.CENTER)


# scale = tk.Scrollbar(frame_lateral, orient=tk.VERTICAL)
# scale.pack()



In [133]:
# # ANÁLISE DE CONFORMIDADE --- BET #

# from pm4py.algo.conformance.tokenreplay import algorithm as token_based_replay
# from pm4py.algo.conformance.tokenreplay.diagnostics import duration_diagnostics

# logTeste = pd.read_excel("C:\\Users\\betsabenogueira\\grupo-2\\Notebooks\\Arquivos\\Relatório_NT_Deles.xlsx")

# logTeste = pm4py.convert_to_event_log(logTeste)

# def analiseConform(log):
#     # Criando modelo de log padrão para ser comparado
#     filtered_log = pm4py.filter_variants_top_k(log, 3)

#     # Aplicando algoritimo indutivo
#     net, initial_marking, final_marking = pm4py.discover_petri_net_inductive(filtered_log)

#     # Replay baseado em token com configurações especiais
#     parameters_tbr = {token_based_replay.Variants.TOKEN_REPLAY.value.Parameters.DISABLE_VARIANTS: True, token_based_replay.Variants.TOKEN_REPLAY.value.Parameters.ENABLE_PLTR_FITNESS: True}
#     replayed_traces, place_fitness, trans_fitness, unwanted_activities = token_based_replay.apply(log, net,
#                                                                                                   initial_marking,
#                                                                                                   final_marking,
#                                                                                                   parameters=parameters_tbr)
    
#     # Análise de rendimento (execução inadequada)
#     trans_diagnostics = duration_diagnostics.diagnose_from_trans_fitness(log, trans_fitness)
#     for trans in trans_diagnostics:
#         print(trans, trans_diagnostics[trans])

# analiseConform(logTeste)

### Botão de aplicar valores (TESTE)

In [134]:
##########################Para a função escolhe arquivo que vai criar um grafico novo##########################
# aplicar = s.CTkButton(frame_lateral, text='Aplicar', command=pega_valor_scale)
# aplicar.pack()

### Criando widgets da lable lateral

In [135]:
# Bet

# módulos
from tkinter import ttk

# funções
def on_click():
    # Alterna o estado do botão
    if combo3.winfo_ismapped():
        #button.config(state=tk.DISABLED)
        label_secon.pack_forget()
        combo3.pack_forget()
        print("Botão ativado")
    else:
        #button.config(state=tk.NORMAL)
        label_secon.pack(padx=0, pady=0)
        combo3.pack(padx=0, pady=0)
        print("Botão desativado")

def selecionar_opcao(event=None):
    opcao_selecionada = combo.get()
    print("Opção Selecionada:", opcao_selecionada)

# funções do botão de frequency/performance
def open_frequency():
    label_perf.pack_forget()
    combo2.pack_forget()
    label_freq.pack(padx=0, pady=0)
    combo.pack(padx=0, pady=0)

def open_performance():
    label_freq.pack_forget()
    combo.pack_forget()
    label_perf.pack(padx=0, pady=0)
    combo2.pack(padx=0, pady=0)
    
# # B.P.
# # LABEL DA PARTE LATERAL INFERIOR (DUDA: COMENTEI PQ NN SEI PQ TEM )
# # label_direita_inf = s.CTkLabel(frame_lateral, padx=60, pady=10, )#borderwidth=2, relief="solid"
# # label_direita_inf.pack(fill=tk.Y, expand=True)#side = 'bottom',

# # FREQUENCY
# label_freq = s.CTkLabel(frame_lateral, text="Show Frequency:")

# opcoes = ["Absolute frequency","Case frequency", "Max. repetitions", "Case coverage"]
# combo = s.CTkComboBox(frame_lateral, values=opcoes, state="readonly")
# combo.bind("<<ComboboxSelected>>", selecionar_opcao)

# button1 = s.CTkButton(frame_lateral, text='Frequency', command=open_frequency)
# button1.pack(side = 'top')#

# # PERFORMANCE
# label_perf = s.CTkLabel(frame_lateral, text="Show Performance:")

# opcoes2 = ["Total duration","Median duration", "Mean duration", "Max. duration","Min. duration"]
# combo2 = s.CTkComboBox(frame_lateral, values=opcoes2, state="readonly")
# combo2.bind("<<ComboboxSelected>>", selecionar_opcao)

# button2 = s.CTkButton(frame_lateral, text='Performance', command=open_performance)
# button2.pack()#side = 'top'

# SECONDARY METRICS
# frame_secon = tk.Frame(label_direita_inf2)
# frame_secon.pack()

label_secon = s.CTkLabel(frame_lateral, text="Secondary:")

opcoes3 = ["Absolute frequency","Case frequency", "Max. repetitions", "Case coverage","Total duration","Median duration", "Mean duration", "Max. duration","Min. duration"]
combo3 = s.CTkComboBox(frame_lateral, values=opcoes3, state="readonly")

button3 = s.CTkButton(frame_lateral, text='Add secondary metrics', command=on_click)
button3.pack()#side = 'top'
# tem que sobrepor pois os itens do frame de cima esta desaparendo

In [136]:
botaoConform = s.CTkButton(frame_lateral, text='Análise de Conformidade', command=lambda: analize_conformidade())
botaoConform.pack()

In [137]:
lableConform = s.CTkLabel(frameConform, text=None, font=('Arial', 20))
lableConform.pack()

# labelVariantes = s.CTkLabel(frame_grafico, text=None, font=('Arial', 20))
# labelVariantes.pack(side='bottom')

## LOOPING DA JANELA PRINCIPAL

In [138]:
raiz.mainloop()

C:\Users\meduarda\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pm4py\utils.py:96: UserWarning: Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.
  warnings.warn("Some rows of the Pandas data frame have been removed because of empty case IDs, activity labels, or timestamps to ensure the correct functioning of PM4Py's algorithms.")
replaying log with TBR, completed traces :: 100%|██████████| 8/8 [00:00<00:00, 435.09it/s]
